# Predictive Maintenance with LSTM Deep Learning Models

### For the end to end original code refer to: https://github.com/Azure/lstms_for_predictive_maintenance

#### Load the necessary library and input data

In [ ]:
# Import necessary components
import os
#os.environ["KERAS_BACKEND"] = "cntk"
import keras
import re
import pandas as pd
import numpy as np
import datetime

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from math import ceil

In [ ]:
# Input the data from blob location - user needs to input Azure blob credentials
aml_dir = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']
my_service = BlockBlobService(account_name=storage_account, account_key=storage_key)
my_service.get_blob_to_path('blob-name', fname, '/azureml-share/file_name.csv')

input_data = pd.read_csv(aml_dir + 'file_name.csv')
# Add in relevant feature engineering to date variables if needed

#### Prepare the input data for the LSTM model

In [ ]:
# Function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]
        
# For your dataset create the sequence_array and label_array
# Split the data into train/test based on timestamp

#### Define LSTM network

In [ ]:
# build the network
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()

model.add(LSTM(80, input_shape=(sequence_length, nb_features), return_sequences=True))
model.add(Dropout(0.9))

model.add(LSTM(40, return_sequences=False))
model.add(Dropout(0.9))

model.add(Dense(nb_out, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print(model.summary())

#### Build and evaluate the model

In [ ]:
%%time
# fit the network
model.fit(seq_array, label_array, nb_epoch=10, batch_size=200, validation_split=0.2, verbose=1, 
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')])

In [ ]:
# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Accuracy: {}'.format(scores[1]))

In [ ]:
# predictions
predicted_prob = model.predict_proba(seq_array,verbose=1, batch_size=200)

In [ ]:
# test metrics
scores_test = model.evaluate(seq_array_test, label_array_test, verbose=2)
print('Accuracy: {}'.format(scores_test[1]))

In [ ]:
y_pred_prob = model.predict_proba(seq_array_test)
y_pred_prob = y_pred_prob/max(y_pred_prob)

# make predictions and compute confusion matrix
y_pred_test = y_pred_prob > 0.03
y_true_test = label_array_test
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true_test, y_pred_test)
cm

In [ ]:
# compute precision and recall
precision_test = precision_score(y_true_test, y_pred_test)
recall_test = recall_score(y_true_test, y_pred_test)
f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
print( 'Precision: ', precision_test, '\n', 'Recall: ', recall_test,'\n', 'F1-score:', f1_test )

#### Save the trained model for future scoring

In [ ]:
# Save the model for operationalization: https://machinelearningmastery.com/save-load-keras-deep-learning-models/
import os
import h5py
from sklearn import datasets 
 
# save model
# serialize model to JSON
model_json = model.to_json()
with open("modellstm.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("modellstm.h5")
print("Model saved")

In [ ]:
with open(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'] + 'modellstm.json', 'wt') as json_file:
    json_file.write(model_json)
    print("json file written shared folder")
    json_file.close()

In [ ]:
model_cp.save_weights(os.path.join(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'], 'modellstm.h5'))

In [ ]:
for name in glob.iglob('modellstm.json'):
    my_service.create_blob_from_path(output_container,'modellstm.json', name)

In [ ]:
for name in glob.iglob('modellstm.h5'):
    my_service.create_blob_from_path(output_container,'modellstm.h5', name)